<a href="https://colab.research.google.com/github/EEarth1270/houseprice-prediction/blob/main/stackedregressor-12-.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,VotingRegressor,StackingRegressor
from sklearn.linear_model import Ridge, LinearRegression, BayesianRidge,ElasticNet,ARDRegression,HuberRegressor,SGDRegressor
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder
from sklearn.model_selection import GridSearchCV
from scipy.stats import iqr
from pandas.api.types import CategoricalDtype 
import numpy as np
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

/usr/lib/python3/dist-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
train_dataset = pd.read_csv('../data/train.csv')

In [ ]:
cols_with_no_nans = []
for col in train_dataset.columns:
  if not train_dataset[col].isnull().any():
    if col != 'SalePrice':
      cols_with_no_nans.append(col)

In [ ]:
salePrice = train_dataset['SalePrice']
train_dataset= train_dataset[cols_with_no_nans]
test_dataset = pd.read_csv('../data/test.csv')
test_dataset= test_dataset[cols_with_no_nans]
dataset =  train_dataset.append(test_dataset)

In [ ]:
def oneHotEncode(df,colName):
    for col in colName:
        if( df[col].dtype == np.dtype('object')):
            dummies = pd.get_dummies(df[col],prefix=col)
            df = pd.concat([df,dummies],axis=1)
            df.drop([col],axis = 1 , inplace=True)
    return df

In [ ]:
dataset = oneHotEncode(dataset,cols_with_no_nans)

In [ ]:
train_dataset = dataset[:1460]
test_dataset = dataset[1460:]
train_dataset = pd.concat([train_dataset,salePrice],axis=1)

In [ ]:

train_dataset['lognorm'] = np.log10(train_dataset['SalePrice'])

In [ ]:
train_ds = train_dataset

In [ ]:
test_dataset.fillna(0,inplace=True)

/home/jupyteruser/.local/lib/python3.8/site-packages/pandas/core/frame.py:4316: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [ ]:
esti  = [('huber',HuberRegressor(epsilon=1.25)),('lr',LinearRegression()),
         ('rf',RandomForestRegressor(n_estimators=1000,max_features=0.33, n_jobs=- 1)),
         ('elas',ElasticNet()),
         ('gb',GradientBoostingRegressor(n_estimators=10000,max_features=0.33,learning_rate=0.02)),
         ('lgb',LGBMRegressor(n_estimators=10000,max_features=0.33,learning_rate=0.02, n_jobs=- 1)),
         ('xgb',XGBRegressor(n_estimators=10000,max_features=0.33,learning_rate=0.02, n_jobs=- 1)),
         ('ridge',Ridge(alpha=8)),('bayesridge',BayesianRidge()),('ard',ARDRegression())]
reg = StackingRegressor(estimators=esti,n_jobs=-1)

In [ ]:
reg.fit(train_ds.iloc[:,1:-2],train_ds['lognorm'])

StackingRegressor(estimators=[('huber', HuberRegressor(epsilon=1.25)),
                              ('lr', LinearRegression()),
                              ('rf',
                               RandomForestRegressor(max_features=0.33,
                                                     n_estimators=1000,
                                                     n_jobs=-1)),
                              ('elas', ElasticNet()),
                              ('gb',
                               GradientBoostingRegressor(learning_rate=0.02,
                                                         max_features=0.33,
                                                         n_estimators=10000)),
                              ('lgb',
                               LGBMRegressor(learning_rate=0.02,
                                             max_features=0.33,
                                             n_estimat...
                                            max_features=0.33,
             

In [ ]:
test_dataset = test_dataset.fillna(0)

In [ ]:
result = reg.predict(test_dataset.iloc[:,1:])

In [ ]:
print(result)

[5.07541951 5.19713942 5.26096686 ... 5.19405197 5.04968579 5.36210569]


In [ ]:
result = 10 ** result 

In [ ]:
print(result)

[118965.08112605 157448.82300098 182375.65338659 ... 156333.47065412
 112120.69697992 230200.19695463]


In [ ]:
ans=pd.DataFrame({'SalePrice':result})
submission = pd.concat([test_dataset['Id'],ans],axis=1)
submission.to_csv('../output_ML/wait_for_score/outputstacking12.csv', index=False)